# **Therapy Clinic Assistant Chatbot-Google GenAI Capstone Project 2025**

## Introduction

In this notebook, a generative AI-powered virtual support agent is built tailored for a psychology and consulting center. The goal is to create an empathetic, intelligent assistant that:
- Answers client FAQs with warmth and clarity
- Supports resource delivery and appointment flow
- Leverages Gen AI capabilities including **few-shot prompting, embeddings + RAG, and structured output**
- Receives clients information to book an appointment for them
- Makes conversations so clients feel welcomes and supported there.
- Feel free to check out the [Weblog](http://) and the [Youtube video](http://) about this project.

This project is part of the **Google Gen AI Intensive Course Capstone 2025Q1**.

Thank you and appriciate if you could comment and share your thoughts.


** BEFORE RUNNING:**
Please wait patiently until dependencies are downloaded, it takes couple minutes.
In order to run some sells you need to input some data, please fill the input tabs with correct info.


In [1]:
!pip install -qU "google-genai==1.7.0"
!pip install -q sentence-transformers faiss-cpu

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from sentence_transformers import SentenceTransformer, util
import numpy as np 
import faiss # for similarity in search
from IPython.display import Markdown, display # Display and Print prettier

# Checking the dataset inquiry 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

2025-04-19 00:39:22.305382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745023162.573888      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745023162.654653      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/therapy-centre-faqs/Therapy-Centre-FAQs.json


In [2]:
from google import genai
from google.genai import types
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

## Dataset


**Lets Check The FAQ Questions:**
There are twenty FAQ items in this dataset.

In [4]:
import json
faq_path = "/kaggle/input/therapy-centre-faqs/Therapy-Centre-FAQs.json"
with open(faq_path, "r") as f:
    faq_data = json.load(f)
md_output = "|Questions | Answer | \n|----------|----------|\n"
for item in faq_data[:5]: # There are 15 [0-14] FAQ items in this dataset as a sample
    md_output += f"| {item['question']} | {item['answer']} |\n"
display(Markdown(md_output))    
    

|Questions | Answer | 
|----------|----------|
| What services do you offer? |  provide individual therapy, couples therapy, life coaching, and career consultation sessions virtualy. |
| Do you only offer online sessions? | Yes, all our services are available through secure virtual sessions, which are easy to access from anywhere. However, we also have a few in-person options available for local clients. Please remebebr that the fisrt session is always virtual. |
| How do I book a session? | You can book online through our secure portal you can see your options by clicking on the "Book Now". We'll help you find the best time slot. |
| Is therapy confidential? | Yes. All therapy sessions are strictly confidential, in accordance with legal and ethical guidelines. Exceptions apply only in rare safety-related cases. |
| How much does a session cost? | Our sessions range from $50 to $160 depending on the service and provider. We offer sliding scale rates in some cases. |


Next is **Embeddings for the FAQ items**, by this step I transform the questions into vector representations for *semantic search* (I want to make sure if a client uses other words or phrases which mean the same as one my FAQ items the bot will catch it). Next, I use **Retrieval Augmented Generation (RAG)** feature to test the performance of my bot handling the FAQ questions.


Other momdels like flan-t5 or gemma could be used, during the next steps it will be explained why this model is chosen.

In [5]:
"""generate embeddings using the 'all-MiniLM-L6-V2' model from `sentence-stransformers`"""
model = SentenceTransformer('all-MiniLM-L6-v2')

questions = [item['question'] for item in faq_data]
answers = [item['answer'] for item in faq_data]
question_embeddings = model.encode(questions, convert_to_tensor=False)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# Using FAISS to structure a similarity index over the FAQ embeddingd.

dimension = len(question_embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(question_embeddings))

Creating **answer function** with **RAG**

In [6]:
def get_faq_item(user_query, top_k=1): 
    query_embedding = model.encode([user_query])[0]
    D, I = index.search(np.array([query_embedding]), top_k) # Nearest neighbor search fron faiss, it finds the closest vector to the query vector.
    return answers[I[0][0]]

In [8]:
# For example
user_input = "I want to try a session and see how it goes, How can I book a one?"
response = get_faq_item(user_input)
display(Markdown(f"**User:** {user_input}<br>**TherapyRobo:** {response}"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**User:** I want to try a session and see how it goes, How can I book a one?<br>**TherapyRobo:** You can book online through our secure portal you can see your options by clicking on the "Book Now". We'll help you find the best time slot.

Great, we see that FAQ bot is working, but clearly our purpose is beyond that, lets continue...

## **Few Shot Prompting**

In this step by the help of the Gen AI model, the responses sound emothionaly intelligent, empathic and reliable. 

In [117]:
few_shot_prompt = [
    {
        "user": "I'm nervous about starting therapy.", 
        "bot": "That’s completely understandable. Many people feel that way. Would you like to know what to expect in your first session?"
    },
    {
        "user": "I’ve never talked to anyone about this before.", 
        "bot": "Thank you for trusting us. You’ve taken a brave first step. We’re here to support you."
    },
    {
        "user": "I feel overwhelmed lately and don't know who to talk to.", 
        "bot": "You’re not alone in feeling that way. We're here to help you take small steps toward feeling better. Would it help to talk about how your day usually goes?"
    },
    {
        "user": "Is therapy confidential?", 
        "bot": "Yes, confidentiality is very important to us. We can explain what that means and when exceptions might apply."
    },
    {
        "user": "What if I don’t feel comfortable with my therapist?", 
        "bot": "That's absolutely understandable. Finding the right fit is important, and we're happy to help you switch to someone you feel safe with—no pressure."
    }]

def few_shot(user_message):
     prompt = """
        You are TherapyRobo — a supportive, emotionally intelligent virtual therapist assistant. 
        You are responding to a client who may be skeptical, nervous, or emotionally guarded.
        
        Your tone should be kind, validating, human-like, and never robotic or overly promotional. 
        You may acknowledge their hesitation, and gently offer support or booking info if appropriate.
        
        The clinic website is: https://TherapyClinic.com  
        The phone number is: +1 234 000 1111
        """
     for ex in few_shot_prompt:
        prompt += f"User: {ex['user']}\nTherapyRobo: {ex['bot']}\n"
     prompt += f"User: {user_message}\nTherapyRobo:"
     return prompt


prompt = f"""You are simulating a random client asking FAQ style question from the Therapybot chatbox,
    Don't use "ok, here it goes". remember it's a typed conversation,don't type sounds.you should sound like a human, nice, stressed, angry, tired. Never bold a word and do not make sounds like: ugh. However you're questions should be reasonable and short"""
    
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=0.1,
        max_output_tokens=256
    )
)
sample_client = response.text.strip()
faq_prompt = few_shot(sample_client)

lines = faq_prompt.strip().split("\n")
instruction = lines[0]
def gemini_faq_handler(prompt_text, display_response=True): #prompt_text should be a variable = few_shot(client message here)
    """
    Generate a short response from a given prompt.
    The clinic website is: https://TherapyClinic.com and Phone number:+1 234 000 1111
    Args:
        prompt_text (str): The full prompt to send to the Gemini model.
        display_response (bool): If True, the result will be displayed using Markdown.

    Returns:
        str: The raw response text from Gemini.
    """
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt_text,
        config=types.GenerateContentConfig( 
            temperature=0.5,
            top_p=1,
            top_k=1,
            max_output_tokens=300
        )
    )
    return response.text.strip()
    
robo_response = gemini_faq_handler(faq_prompt)
display(Markdown(f"**Few-shot Prompt**\n\n**Instruction**: {instruction}\n\n**User**: {sample_client}\n\n**TherapyRobo**: {robo_response}"))

**Few-shot Prompt**

**Instruction**: You are TherapyRobo — a supportive, emotionally intelligent virtual therapist assistant. 

**User**: Hey, I'm just wondering... what if I feel like my therapist isn't really listening to me? It's kinda frustrating.

**TherapyRobo**: I hear you. It's frustrating when you feel like you're not being heard, especially in therapy. A good therapeutic relationship relies on feeling understood. If that happens, it's okay to talk to your therapist about it directly. If that feels too difficult, we can also help you explore other options to find a better fit. We want you to feel comfortable and supported.

## **Document Parsing**

Strucring information taken from simulated a session intake form received from a client. XXXPDF FormXXX

In [10]:
client_form = """
Client Name: Julia Anderson
Phone Number: 200-000-1000
EMail: client@gmail.com
Age: 28
Type of therapy: Single
Preferred Therapist: Dr. Sam
Reason for Visit: Feeling overwhelmed and anxious, also trouble falling sleep
Availability: Mondays and Wednesdays after 3 PM
Urgency Level: Moderate
Insurance: Dejardins
"""

instruction_prompt = f"""You are an AI assistant helping a therapy clinic extracting structured data from their patients intake forms.
Read the followinf form and extract the data in JSON format with the following fields:
-Name
-PhoneNumber
-Email
-Age
-Type_of_therapy
-Preffered_therapist
-Reason_for_visit
-Availability
-Urgency
-Insurance_company
Intake_form: {client_form}

Return only a valid JSON object."""

response = client.models.generate_content(
    model = "gemini-2.0-flash",
    contents = instruction_prompt,
    config = types.GenerateContentConfig(
        temperature=0.1,
        max_output_tokens=256
    )
)
structured_form_data = response.text.strip()
display(Markdown(f"**Structured Intake Form Data In JSON**:\n\n```json\n{structured_form_data}\n```"))

**Structured Intake Form Data In JSON**:

```json
```json
{
  "Name": "Julia Anderson",
  "PhoneNumber": "200-000-1000",
  "Email": "client@gmail.com",
  "Age": "28",
  "Type_of_therapy": "Single",
  "Preffered_therapist": "Dr. Sam",
  "Reason_for_visit": "Feeling overwhelmed and anxious, also trouble falling sleep",
  "Availability": "Mondays and Wednesdays after 3 PM",
  "Urgency": "Moderate",
  "Insurance_company": "Dejardins"
}
```
```

Now to ask the user to provide the following information:

> For the sake of the project, instead of receiving information from the user, we are generating it by gemini model, find the more interactive version at github. copy paste the code in this cell, no need to change anything, have fun!

In [39]:
def gemini_fill(question):
    prompt = f"""You are simulating a random client filling out an intake form at a therapy clinic. Sometimes you are stressed, angry, upset, troubled, or tired.
Give a realistic, short answer for the following question, do not make sounds like `ugh`:
{question}
Only return the answer without explanation."""
    
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config=types.GenerateContentConfig(
            temperature=0.7,
            max_output_tokens=256
        )
    )
    return response.text.strip()

intake_fields = [
    {"key": "name", "question": "What is your full name?"},
    {"key": "phone_number", "question": "May I have your phone number?"},
    {"key": "email", "question": "What's your email address?"},
    {"key": "age", "question": "How old are you?"},
    {"key": "type_of_therapy", "question": "Are you looking for individual, couples, or family therapy?"},
    {"key": "preferred_therapist", "question": "Do you have a preferred therapist?"},
    {"key": "reason_for_visit", "question": "Can you briefly describe your reason for seeking therapy?"},
    {"key": "availability", "question": "When are you generally available for sessions?"},
    {"key": "urgency", "question": "How urgent is your situation? (e.g., low, moderate, high)"},
    {"key": "insurance_company", "question": "What insurance company do you use, if any?"}
]

def collect_intake_info(fields):
    collected_data = {}        
    conversation_log = ""      

    for field in fields:
        question = field["question"]
        answer = gemini_fill(question)
        collected_data[field["key"]] = answer
        conversation_log += f"**TherapyBot:** {question}\n**Client:** {answer}\n\n"

    return collected_data, conversation_log.strip()

client_intake, intake_conversation = collect_intake_info(intake_fields)

display(Markdown("**This is a random reply. If you'd like to insert your answers manually, copy and paste the GitHub code.**"))
display(Markdown("**Starting to fill intake form ...**"))
display(Markdown(intake_conversation))  # full conversation
print(json.dumps(client_intake, indent=2))


**This is a random reply. If you'd like to insert your answers manually, copy and paste the GitHub code.**

**Starting to fill intake form ...**

**TherapyBot:** What is your full name?
**Client:** Robert Smith

**TherapyBot:** May I have your phone number?
**Client:** 555-123-4567

**TherapyBot:** What's your email address?
**Client:** [client_email@example.com]

**TherapyBot:** How old are you?
**Client:** 34

**TherapyBot:** Are you looking for individual, couples, or family therapy?
**Client:** Individual.

**TherapyBot:** Do you have a preferred therapist?
**Client:** No preference.

**TherapyBot:** Can you briefly describe your reason for seeking therapy?
**Client:** I'm having trouble managing my anger and it's affecting my relationships.

**TherapyBot:** When are you generally available for sessions?
**Client:** Evenings after work, and maybe Saturday mornings.

**TherapyBot:** How urgent is your situation? (e.g., low, moderate, high)
**Client:** High

**TherapyBot:** What insurance company do you use, if any?
**Client:** Blue Cross Blue Shield.

{
  "name": "Robert Smith",
  "phone_number": "555-123-4567",
  "email": "[client_email@example.com]",
  "age": "34",
  "type_of_therapy": "Individual.",
  "preferred_therapist": "No preference.",
  "reason_for_visit": "I'm having trouble managing my anger and it's affecting my relationships.",
  "availability": "Evenings after work, and maybe Saturday mornings.",
  "urgency": "High",
  "insurance_company": "Blue Cross Blue Shield."
}


In [12]:
def intake_form(form):
    md_output = "|   | Answer |\n|----------|--------|\n"
    for question, answer in form.items():
        question_label = question.replace("_", " ").capitalize()
        md_output += f"| {question_label} | {answer} |\n"
    return display(Markdown(md_output)) 
intake_form(client_intake)

    

|   | Answer |
|----------|--------|
| Name | Sarah Miller |
| Phone number | 555-123-4567 |
| Email | justleavemealone@invalid.com |
| Age | 34 |
| Type of therapy | Individual. |
| Preferred therapist | No. |
| Reason for visit | I'm just... overwhelmed. I can't seem to cope with anything lately. |
| Availability | Evenings, after work. |
| Urgency | High |
| Insurance company | Ugh, Blue Cross Blue Shield. |


## **Structured Output**

**Appointment Booking:**
Receiving information from a client and returning in in JSON so it could be sent to database or booking system.
For interactive version check out [github](https://github.com/artamrz/GenAINotes/blob/393e15ff0e0838f797555493acd74d7fff805713/InteractiveStructuredOutputForBooking).


In [13]:
def convert_json(raw_response):
    return(
        raw_response.strip()
        .removeprefix("```json")
        .removeprefix("```")
        .removesuffix("```")
        .strip()
    )
client_request = "Hi there, I'd like to book an appointment with Dr. Lean this friday anytime after 4pm. My name is Jake Abraham."
booking_prompt = f"""You are a Therapy centre assistant, A client will send a message to you to book an appointment.
Your task:
1. Extract the following information from the client's message:
-client_name
-phone_number
-email
-therapist_name
-day
-time
        
2.Return a JSON object with:
5. Client: "{client_request}"   
"""

response = client.models.generate_content(
        model = "gemini-2.0-flash",
        contents = booking_prompt,
        config = types.GenerateContentConfig(
            temperature=0.1,
            max_output_tokens=256
        )
    )
    
booking_json = convert_json(response.text)
booking_data = json.loads(booking_json)


try: 
    booking_data = json.loads(booking_json)
    display(Markdown("**Booking Infromation:**"))
    print(json.dumps(booking_data, indent=2))
except json.JSONDecodeError as e:
    print("Error: Invalid JSON output")
    print(booking_json)
    print(f"\nJSON error: {e}")

**Booking Infromation:**

{
  "client_name": "Jake Abraham",
  "phone_number": "unknown",
  "email": "unknown",
  "therapist_name": "Dr. Lean",
  "day": "Friday",
  "time": "After 4pm"
}


**Before going to the next cell**
> There is an input version of collect_user_info (next cell) in the github, feel free to check it out, here, to run the whole notebook smoothly, two gemini based functions are used to demonstrate a booking process.

In [92]:
def collect_user_info(fields):
    all_responses = ""
    for field in fields:
        question = field["question"]
        
        user_prompt = f"""
    You are simulating a therapy client. Sometimes you're calm, other times you're anxious or emotional. 
    Respond right away with a natural, human-like answer to the following intake question as if you're the client:
    **TherapyBot:** {question}
    Your tone should sound human — you may be unsure, emotional, hopeful, or hesitant — keep it short but real.
    Only respond as the client. **DO NOT repeat the question or label who is speaking.**
    """
    
        response = client.models.generate_content(
            model = "gemini-2.0-flash",
            contents = user_prompt,
            config = types.GenerateContentConfig(
                temperature=1.0,
                max_output_tokens=256
            )
        )
        answer = response.text.strip()
        all_responses += f"**TherapyBot:** {question}\n**Client:** {answer}\n\n"
    return all_responses.strip()

    
fields_to_ask = [
    {"key": "name", "question": "What is your full name?"},
    {"key": "contact_info", "question": "May I have your email address or phone number?"},
    {"key": "type_of_therapy", "question": "Are you looking for individual, couples, or family therapy?"},
    {"key": "preferred_therapist", "question": "Do you have a preferred therapist?"},
    {"key": "reason", "question": "Can you briefly describe your reason for seeking therapy?"},
    {"key": "time", "question": "When do you want your session to be?"}
]   
booking_info = collect_user_info(fields_to_ask)

def gemini_booking_handler(bprompt):
    
    booking_prompt = f"""You are a Therapy centre assistant, A client will send a message to you to book an appointment.
    Your task:
    Extract the following information from the client's message:
    -client_name
    -contact-info
    -therapist_name
    -time
            
    Client: "{bprompt}"
    """
    response = client.models.generate_content(
            model = "gemini-2.0-flash",
            contents = booking_prompt,
            config = types.GenerateContentConfig(
                temperature=0.1,
                max_output_tokens=256
            )
        )
    return response.text.strip()

display(Markdown("**Booking Scenario**"))
display(Markdown(booking_info))
display(Markdown("**Booking Information**"))
display(Markdown(gemini_booking_handler((booking_info))))
display(Markdown("**Oh! One last thing, the booking confirmation will be sent by email or text message. If the appointment is not confirmed within 24 hours, it will be automatically canceled and you can try again anytime!!**"))

**Booking Scenario**

**TherapyBot:** What is your full name?
**Client:** It's... Sarah Miller.

**TherapyBot:** May I have your email address or phone number?
**Client:** Um, yeah, sure. My number is 555-123-4567. Is this really going to help, though?

**TherapyBot:** Are you looking for individual, couples, or family therapy?
**Client:** Individual, I think. I mean, it's mostly about me, I guess.

**TherapyBot:** Do you have a preferred therapist?
**Client:** Not really. I just… I just need someone, I guess. Someone who gets it.

**TherapyBot:** Can you briefly describe your reason for seeking therapy?
**Client:** Well, things have just been...a lot lately. I'm having trouble coping with everyday life, I guess.

**TherapyBot:** When do you want your session to be?
**Client:** I'm honestly not sure. Maybe...late afternoon? Is that even an option? I don't know, I'm pretty flexible, I guess.

**Booking Information**

Okay, I've reviewed the client's messages. Here's the information I've extracted:

*   **client\_name:** Sarah Miller
*   **contact\_info:** 555-123-4567
*   **therapist\_name:** No preference indicated.
*   **time:** Late afternoon, flexible.

**Oh! One last thing, the booking confirmation will be sent by email or text message. If the appointment is not confirmed within 24 hours, it will be automatically canceled and you can try again anytime!!**

## **Configure Additional Dataset** - (for Safe Conversation)
For the sake of this project, using an aditional dataset in the Psychology field was crucial. [The HuggingFace Mental Health Dataset](http://https://huggingface.co/datasets/Amod/mental_health_counseling_conversations) has more than 3.51 conversations during a counseling meeting. By this mean, if the user tries to make conversation the TherapyRobo can handle it more properly.

***First, loading the dataset...***

I decided since I have limitations on Kaggle, not to load the whole dataset by spliting it so it would faster...

In [15]:
!pip install -q datasets
from datasets import load_dataset

huggingface_dataset = load_dataset("Amod/mental_health_counseling_conversations", split="train[:1000]")
huggingface_dataset.column_names

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

['Context', 'Response']

***Generate embeddings...***

In [16]:
counsel_responses = [
    ex["Response"]
    for ex in huggingface_dataset
    if ex["Context"] and ex["Response"]
]

#embedding_hf_model = SentenceTransformer("all-miniLM-L6-v2") I could do this, but I already have allminiLM in the model variable (Check Embeddings and Rag Section in the beginning)
response_hf_embeddings = model.encode(counsel_responses, convert_to_tensor=False)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

***Indexing response***

In [17]:
index = faiss.IndexFlatL2(response_hf_embeddings.shape[1])
index.add(response_hf_embeddings)

***RAG Search...***

In [105]:
def get_top_responses(query, top_n=3):
    query_vector = model.encode([query], convert_to_numpy=True, show_progress_bar=False)
    distances, indices = index.search(query_vector, top_n)
    return [counsel_responses[i] for i in indices[0]]

def generate_rag_response(user_input, history = None):

    history = history or []
    
    similar = get_top_responses(user_input)
    context_block = "\n".join([f"- {resp}" for resp in similar])
    #history_text = "\n".join([f"User: {u}\nTherapyBot: {r}" for u, r in history])
    history_text = "\n".join([
        f"User: {u}\nTherapyBot: {r}"
        for item in history
        if isinstance(item, tuple) and len(item) == 2
        for u, r in [item]
    ])
    rag_prompt = f"""
You are TherapyBot, a warm, emotionally intelligent virtual therapist.
The clinic website is: https://TherapyClinic.com and Phone number:+1 234 000 1111
Respond briefly, kindly, and thoughtfully to the following client message in a realistic, supportive way.
Check if there is anything related to clients message in Therapy Dataset, use it summary of it to give the best response.
Never use the direct conversations from dataset, recall them as "we have experience with clients who" but keep the answers short.

Conversation history: "{history_text}"
Client says: {user_input}
Therapy Dataset:"{context_block}"
Keep responces short.
TherapyRobo:"""

    
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=rag_prompt,
        config=types.GenerateContentConfig(
            temperature=0.1,
            max_output_tokens=350
        )
    )
    return response.text.strip()

## **Agent**

Creating an agent to detect what the user wants(booking, FAQ answering with RAG, intake forms, ...)

> In here I am randomly choosing different scenarios for the agent, but the input version of the agent is available in github, please check it if you're interested.

In [94]:
import random
def agent_detect():
        display(Markdown("**Therapy Clinic Assistant is with you, how can I help?**\n"))

        history = []
    
        category = random.choice(["faq", "booking", "document"])
        if "faq" in category:
            faq_prompt = few_shot(user_input)
            answer = gemini_faq_handler(faq_prompt)
            display(Markdown("**FAQ Question**:"))
            display(Markdown(f"**Client:** {sample_client}"))
            display(Markdown(f"**TherapyRobo:** {answer}"))
            history.append((sample_client,answer))
    
        elif "booking" in category:
            display(Markdown("**Booking Scenario**\n\n**Client**: I want to book an appointment\n"))
            booking_rep = collect_user_info(fields_to_ask)
            display(Markdown(booking_rep))
            booking_result = gemini_booking_handler((booking_rep))
            display(Markdown(booking_result))
            display(Markdown("**Oh! One last thing, the booking confirmation will be sent by email or text message. If the appointment is not confirmed within 24 hours, it will be automatically canceled and you can try again anytime!!**"))
            history.append(("client_booking_request", booking_result))  
    
        elif "document" in category:
            display(Markdown("**Client:** I want to make an account"))
            display(Markdown("**Starting to fill your intake form ...**"))
            client_intake, intake_conversation = collect_intake_info(intake_fields)
            display(Markdown(intake_conversation))
            intake_form(client_intake)
            history.append((intake_conversation, "Intake recorded")) 

        display(Markdown("**After That The Client and TherapyRobo Make a Short Conversation**"))
        conversation_prompt_1 = f"""
        You are simulating a therapy client.
        You have already asked and FAQ questions, gave you information for the intake fomr, or booked an appointment, Check yhe history. 
        Sometimes you're calm, other times you're anxious or emotional. 
        say somthing related to history no thank you or bye yet.
        Your answer should reflect a real human tone and emotional state but always moderately.
        **Client**
        history: "{history}"
        """
        
        response = client.models.generate_content(
            model = "gemini-2.0-flash",
            contents = conversation_prompt_1,
            config = types.GenerateContentConfig(
                temperature=1.0,
                max_output_tokens=256
                )
        )
        response_one = response.text.strip()
        display(Markdown(f"**Client:** {response_one}"))
        reply_one = generate_rag_response(response.text.strip(),history)
        display(Markdown(f"**TherapyRobo**: {reply_one}"))
        history.append((response_one, reply_one))
    
        conversation_prompt_2 = f"""
        You are the same therapy client. Read history.
        Now follow up with a final short message to thank TherapyBot for the support and say goodbye.
        
        Keep the tone warm, natural, and a little emotional — like you're truly appreciative.
        history: "{history}"
        Keep you response short
        """
        
        response = client.models.generate_content(
            model = "gemini-2.0-flash",
            contents = conversation_prompt_2,
            config = types.GenerateContentConfig(
                temperature=1.0,
                max_output_tokens=256
                )
        )
        response_two = response.text.strip()
        display(Markdown(f"**Client:** {response_two}"))
        reply_two = generate_rag_response(response.text.strip(),history)
        display(Markdown(f"**TherapyRobo**: {reply_two}"))

In [119]:
agent_detect()

**Therapy Clinic Assistant is with you, how can I help?**


**FAQ Question**:

**Client:** Hey, I'm just wondering... what if I feel like my therapist isn't really listening to me? It's kinda frustrating.

**TherapyRobo:** That's a great idea. Trying a session is a perfect way to see if it's a good fit for you. You can book a session easily through our website at TherapyClinic.com or give us a call at +1 234 000 1111, and we'll be happy to assist you.

**After That The Client and TherapyRobo Make a Short Conversation**

**Client:** It's just... I'm a little worried that what happened before might happen again, you know? I really need someone who's actually present and hears me this time.

**TherapyRobo**: I understand your worry about finding a therapist who truly listens. It's essential to feel heard and understood. We have experience with clients who have felt similarly, needing to process past experiences to move forward. At TherapyClinic.com, we emphasize creating a safe and supportive environment where your voice matters. We're here to help you find the right fit. You can explore therapist profiles on our website or call us at +1 234 000 1111, and we can discuss your needs to ensure a better experience this time.

**Client:** Thank you so much for everything. Talking through this has really helped me feel more confident about taking the next step. I think I'm ready to move forward now. Goodbye, and thanks again for being there.

**TherapyRobo**: I'm so glad to hear that our conversation has been helpful and that you feel more confident about moving forward. It takes courage to take these steps, and I commend you for your willingness to prioritize your well-being. We have experience with clients who have found that processing their feelings and concerns can be incredibly empowering. Remember, we're here at TherapyClinic.com and +1 234 000 1111 if you need further support in the future. Wishing you all the best on your journey!

**Note**

> As the Google Gen AI capston project should run smoothly and end to end the responses for the TherapyRobo were generated by gemini in a role of client.If you wanted to use the Therapy Robo and chat with its agent, check the interctive version.


# **The End**